## Importing Necessary Modules

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import scipy as sc
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def file_reader(data):
    return pd.read_csv(data)

In [ ]:
train = file_reader('Train.csv')
test = file_reader('Test.csv')
variable = file_reader('VariableDefinitions (1).csv').T

In [ ]:
#Creating a Sample of the Whole DataSet for EDA and Feature Engineering

#I had to do this to reduce computation 

#Since i was using a cpu and not Google Colab or Other Kernels that give out free GPU Usage

#df =train.sample(frac= 0.15).reset_index()

In [ ]:
#This Column should be run after you have finish exploring the dataset and creating feature for ease of computation

#when running this column do not run the column of the sample above hence comment the column to prevent change in variable name

df = pd.concat([train,test],ignore_index=True, sort=True)

In [ ]:
#The Varibale name of each of the column to

variable

In [ ]:
def churn_percentage(df, data):
    '''
    To Check if the newly created train sample captures 
    both the Churned and Not Churned in the respected ratio
    '''
    data = data['CHURN'].value_counts()
    data_0 = data[0]
    data_1 = data[1]
    dataset = 100 * (data_1 /(data_0 + data_1))
    
    df= df['CHURN'].value_counts()
    df_0 = df[0]
    df_1 = df[1]
    sample_df = 100 * (df_1 /(df_0 + df_1))
    
    print(f'Percentage of Churned of Sample data is {sample_df}\n\nWhile Percentage of Churned of Whole Datasetet is {dataset} ')


In [ ]:
churn_percentage(df, train)

## Exploratory Data Analysis (EDA)

In [ ]:
#Checking for duplicated columns and rows
df.duplicated().sum()

In [ ]:
#Shape of the data
def shape(data):
    print(f'This is the shape of the df dataset: {data.shape}')

In [ ]:
df.columns

In [ ]:
shape(df)

In [ ]:
#Checking out the Categorical Column and Numerical in the Dataset
def cat_num(data):
    cat_col = []
    num_col = []
    for feat in data.columns:
        if data[feat].dtypes =='O':
            cat_col.append(feat)
        else:
            num_col.append(feat)
    return cat_col, num_col

In [ ]:
#This columns where dropped because there are all indentical
df = df.drop(labels = ['user_id','MRG'], axis= 1)

In [ ]:
def min_max(column):
    print(f'Max {df[column].max()}\n\nMin {df[column].min()}')

In [ ]:
min_max("REVENUE")

In [ ]:
df.TENURE.value_counts()

In [ ]:
def time_frame(data):
    new_var = []
    new_list = [i for i in data['TOP_PACK']]
    for val in new_list:
        val = str(val)
        if '_' and ';' in val:
            new_var.append(val.split(';')[-1])
        elif '_' in val:
            new_var.append(val.split('_')[-1])
        elif ';' in val:
            new_var.append(val.split(';')[-1])
        elif ',' in val:
            new_var.append(val.split(',')[-1])
        elif 'nan' in val:
            new_var.append(np.nan)
        else:
            new_var.append(val)
    return new_var

In [ ]:
df['TIME_FRAME'] = time_frame(df)

In [ ]:
df['TIME_FRAME'] = df['TIME_FRAME'].str.lower()

In [ ]:
def data_call(data):
    data_type = [i for i in data['TIME_FRAME']]
    data_cal = []
    for val in data_type:
        val = str(val)
        if 'call' in val:
            data_cal.append(1)
        elif 'nan' in val:
            data_cal.append(np.nan)
        else:
            data_cal.append(0)
    return data_cal

In [ ]:
def unlimited(data):
    limi =[]
    __ = [i for i in data['TOP_PACK']]
    for value in __:
        value = str(value)
        if 'limi' in value:
            limi.append(1)
        elif 'nan' in value:
            limi.append(np.nan)
        else:
            limi.append(0)
    return limi

In [ ]:
df['DATA_CAPPED'] unlimited(df)

In [ ]:
def data_type_fix(data):
    data_type = [i for i in df['TOP_PACK'].str.split().str[0].str.lower()]
    fix_data = []
    for value in data_type:
        value = str(value)
        if 'data' in value:
            fix_data.append('data')
        elif 'on' in value:
            fix_data.append('on-net')
        elif 'wifi' in value:
            fix_data.append('wifi')
        elif 'all-net' in value:
            fix_data.append('all-net')
        elif 'nan' in value:
            fix_data.append(np.nan)
        elif 'ivr' in value:
            fix_data.append('var_ivr')
        elif 'twter' in value:
            fix_data.append('twter')
        elif 'jokk' in value:
            fix_data.append('jokko')
        elif 'mixt' in value:
            fix_data.append('mixt')
        else:
            fix_data.append('others')
    return fix_data

In [ ]:
df['DATA_CALL'] = data_call(df)

In [ ]:
df['DATA_TYPE'] = data_type_fix(df)

In [ ]:
def time_range(data):
    data_type = [i for i in data['TIME_FRAME']]
    fix_data = []
    for val in data_type:
        val = str(val)
        if 'd' in val[-1] and 'i' not in val:
            fix_data.append(int(val[:-1]))
        elif 'mo' in val:
            fix_data.append(30)
        elif 'wee' in val:
            fix_data.append(7)
        elif '48h' == val:
            fix_data.append(2)
        elif 'nan' in val:
            fix_data.append(np.nan)
        elif '4h' in val or 'da' in val or '0h' in val:
            fix_data.append(1)
        else:
            fix_data.append(4)
    return fix_data

In [ ]:
df['TIME_RANGE'] = time_range(df)

In [ ]:
#Time Rate

def time_rate(data):
    rate =[]
    __ = [i for i in data['TIME_FRAME']]
    for val in __:
        val = str(val)
        if 'h' in val:
            rate.append(1)
        elif 'dai' in val:
            rate.append(2)
        elif 'wee' in val:
            rate.append(3)
        elif 'mon' in val:
            rate.append(5)
        elif 'nan' in val:
            rate.append(np.nan)
        elif 'd' in val and 'i' not in val:
            rate.append(4)
        else:
            rate.append(0)
    return rate

In [ ]:
df['DATA_RATE'] = time_rate(df)

### Converting the Categorical column to Numeric Values using Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df.REGION = df.REGION.fillna('unknown')
df.DATA_TYPE = df.DATA_TYPE.fillna('unknown')

In [ ]:
df['REGION'] = le.fit_transform(df.REGION).astype(np.uint8)
df.TENURE = le.fit_transform(df.TENURE).astype(np.uint8)
df.DATA_TYPE = le.fit_transform(df.DATA_TYPE).astype(np.uint8)

In [ ]:
df.to_csv('new_dataset.csv', index=False)

<h3>Possible Improvement The Region Column would be better to use Pandas Get Dummy Mehthod</h3>

<b>I notice this late towards the end of the competition and decided to stick with the Label Encoder</b>